## requirements:
- tensorflow (latest)
- nltk
- sklearn
- numpy 
- pandas
- seaborn
- matplotlib

In [1]:
!pip install -q -U tensorflow-text
!pip install -q tf-models-official

In [2]:
!pip install nltk

In [3]:
import os
import shutil
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from official.nlp import optimization  # to create AdamW optimizer

import warnings
import matplotlib.pyplot as plt
import seaborn as sns
tf.get_logger().setLevel('ERROR')

In [4]:
main_data=pd.read_csv("GEN-sarc-notsarc.csv")
data=main_data.copy()
data.drop(columns=['id'],axis=1,inplace=True)
classes = {"notsarc" : 0,"sarc" : 1}
data["class"] = data["class"].map(classes)
data

,class,text
0,0,"If that's true, then Freedom of Speech is doom..."
1,0,Neener neener - is it time to go in from the p...
2,0,"Just like the plastic gun fear, the armour pie..."
3,0,So geology is a religion because we weren't he...
4,0,Well done Monty. Mark that up as your first ev...
...,...,...
6515,1,depends on when the baby bird died. run alon...
6516,1,"ok, sheesh, to clarify, women who arent aborti..."
6517,1,so.. eh?? hows this sound? will it fly w...
6518,1,"I think we should put to a vote, the right of ..."


In [5]:
X=data['text']
y=data['class']
X=np.array(X)
y=np.asarray(y).astype('int32')#.reshape((-1,1))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
print('Train data len:'+str(len(X_train)))
print('Class distribution\n'+str((pd.Series(y_train)).value_counts()))
print('Test data len:'+str(len(X_test)))
print('Class distribution\n'+str((pd.Series(y_test)).value_counts()))

Train data len:5216
Class distribution
1    2616
0    2600
dtype: int64
Test data len:1304
Class distribution
0    660
1    644
dtype: int64


In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [9]:
print('Train data len:'+str(len(X_train)))
print('Class distribution\n'+str((pd.Series(y_train)).value_counts()))
print('Valid data len:'+str(len(X_valid)))
print('Class distribution\n'+str((pd.Series(y_valid)).value_counts()))
print('Test data len:'+str(len(X_test)))
print('Class distribution\n'+str((pd.Series(y_test)).value_counts()))

Train data len:4172
Class distribution
0    2087
1    2085
dtype: int64
Valid data len:1044
Class distribution
1    531
0    513
dtype: int64
Test data len:1304
Class distribution
0    660
1    644
dtype: int64


In [10]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [11]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [12]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [13]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [14]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
    return tf.keras.Model(text_input, net)

In [15]:
classifier_model = build_classifier_model()

In [16]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

In [17]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32)
valid_ds = tf.data.Dataset.from_tensor_slices((X_valid, y_valid)).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

In [18]:
train_ds

<BatchDataset shapes: ((None,), (None,)), types: (tf.string, tf.int32)>

In [19]:
epochs = 2
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [20]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [21]:
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x=train_ds,
                               validation_data=valid_ds,
                               epochs=epochs)

Training model with https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Epoch 1/2
131/131 [==============================] - 1526s 12s/step - loss: 0.5381 - binary_accuracy: 0.7145 - val_loss: 0.4773 - val_binary_accuracy: 0.7519
Epoch 2/2
131/131 [==============================] - 1515s 12s/step - loss: 0.3787 - binary_accuracy: 0.8274 - val_loss: 0.5254 - val_binary_accuracy: 0.7586


In [22]:
loss, accuracy = classifier_model.evaluate(test_ds)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

41/41 [==============================] - 130s 3s/step - loss: 0.4673 - binary_accuracy: 0.7776
Loss: 0.4672989249229431
Accuracy: 0.7776073813438416


In [24]:
model_name='BERT_Base_Sarcasm_GEN_'+str(accuracy)
classifier_model.save(model_name, include_optimizer=True)

In [25]:
test_data=pd.read_csv("HYP-sarc-notsarc.csv")
test_data.drop(columns=['id'],axis=1,inplace=True)
classes = {"notsarc" : 0,"sarc" : 1}
test_data["class"] = test_data["class"].map(classes)
test_data

,class,text
0,0,have no predators to fear? check. who said we ...
1,0,2 hours? damn! that book took me a good 2 day...
2,0,you never played myst? damn!!! i must be reall...
3,0,"Well, if Genesis was in fact true, then we wou..."
4,0,Just making sure that everybody is aware of hi...
...,...,...
1159,1,you really believed me? wow! i never knew i ha...
1160,1,please tell me you're kidding. these bowling b...
1161,1,you're kidding. just because your life is 'a f...
1162,1,the evidence that is provided to you is not en...


In [26]:
test_X=test_data['text']
test_y=test_data['class']
test_X=np.array(test_X)
test_y=np.asarray(test_y).astype('int32')

In [27]:
test_tf = tf.data.Dataset.from_tensor_slices((test_X, test_y)).batch(32)

In [28]:
loss, accuracy = classifier_model.evaluate(test_tf)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

37/37 [==============================] - 116s 3s/step - loss: 0.6971 - binary_accuracy: 0.6813
Loss: 0.6971287131309509
Accuracy: 0.681271493434906


In [29]:
test_data2=pd.read_csv("RQ-sarc-notsarc.csv")
test_data2.drop(columns=['id'],axis=1,inplace=True)
classes = {"notsarc" : 0,"sarc" : 1}
test_data2["class"] = test_data2["class"].map(classes)
test_data2

,class,text
0,0,"Archie, the ONLY issue that gays don't have a ..."
1,0,"No, not really. All that is different is the n..."
2,0,It's ashame that everyone keeps looking for th...
3,0,"Almost? Usually, that is true, and it involves..."
4,0,And so have animals. Plants have been wiped ou...
...,...,...
1697,1,"Tell me genius, how is me accurately and corre..."
1698,1,So you think it is a good idea for public scho...
1699,1,"Now settle down charlie, and try to think rati..."
1700,1,The VPC has a political agenda. The FBI? That ...


In [30]:
test2_X=test_data2['text']
test2_y=test_data2['class']
test2_X=np.array(test2_X)
test2_y=np.asarray(test2_y).astype('int32')

In [31]:
test2_tf = tf.data.Dataset.from_tensor_slices((test2_X,test2_y)).batch(32)
loss, accuracy = classifier_model.evaluate(test2_tf)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

54/54 [==============================] - 169s 3s/step - loss: 0.4811 - binary_accuracy: 0.7885
Loss: 0.4811398386955261
Accuracy: 0.7884841561317444
